In [7]:
import torch
from torch import nn
import torchvision.models as models
import torchvision
import tqdm
import os
import pandas as pd
import numpy as np

In [2]:
dataset_path = "/content/drive/MyDrive/competitions/digit_recognizer/data" # @param {type:"string"}
batch_size = 1024
test_batch_size = 1
n_epoch = 256

if not os.path.exists(dataset_path): # check dataset_path
  print('invalid dataset_path')


In [3]:
class datasets_from_csv():
  def __init__(self, dataset_file): # read digit data from a csv file
    data = pd.read_csv(dataset_file)
    
    if 'label' in data: # if the dataset is train data
      self.dataset_type = 'train'
      self.inputs = torch.tensor(data.drop('label', axis=1).values).reshape([-1,28,28])
      self.target = torch.tensor(data['label'].values)
    else: # if the dataset is test data
      self.dataset_type = 'test'
      self.inputs = torch.tensor(data.values).reshape([-1,28,28])

  def __getitem__(self, idx):
    if self.dataset_type == 'train':
      return (self.inputs[idx], self.target[idx])
    else:
      return self.inputs[idx]
  
  def __len__(self):
    return len(self.inputs)
    

def make_dataloaders(train_dataset, test_dataset):
  trainloader = torch.utils.data.DataLoader(train_dataset, pin_memory=True, batch_size=batch_size, shuffle=True, drop_last=False)
  testloader = torch.utils.data.DataLoader(test_dataset, pin_memory=True, batch_size=test_batch_size, shuffle=False, drop_last=False)

  return trainloader, testloader

In [4]:
train_data_file = os.path.join(dataset_path, 'train.csv')
test_data_file = os.path.join(dataset_path, 'test.csv')

train_dataset = datasets_from_csv(train_data_file)
test_dataset = datasets_from_csv(test_data_file)

train_loader, test_loader = make_dataloaders(train_dataset, test_dataset)

In [6]:
class ResNet(nn.Module):
  def __init__(self, input_channel=1, output_channel=10, channels=[16,16,16,16,32,32,32,32,64,64,64,10]):
    super(ResNet, self).__init__()

    next_channel = input_channel
    layer_cnt = 0
    self.layers = []

    for out_channel in channels:
      self.layers.append(nn.Sequence(
          nn.Conv2d(next_channel, out_channel, kernel_size=3, padding=1),
          nn.BatchNorm2d(16)
      ))
      next_channel = out_channel
    
    self.pool = nn.MaxPool2d(2)
    self.global_avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.activation = nn.ReLU()

  
  def forward(self, inputs):
    for i, layer in enumerate(self.layers):
      inputs = layer(inputs)
      inputs = self.activation(inputs)
      if i%4 == 0:
        inputs = self.pool(inputs)
    
    inputs = self.global_avgpool(inputs)
      

    return inputs


In [ ]:
def train(model, train_loader, test_loader, n_epoch,):
  for epoch in tqdm(range(n_epoch)):
    for data in train_loader:
      inputs, target = data1
